<a href="https://colab.research.google.com/github/vvctort1/Analise-de-Dados-Automatizada/blob/main/AnaliseAutomatizada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vega_datasets

In [ ]:
import pandas as pd
import numpy as np
from vega_datasets import data
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import base64
from io import BytesIO
import os
from jinja2 import Environment, FileSystemLoader


df = data.jobs()

,job,sex,year,count,perc
0,Accountant / Auditor,men,1850,708,0.000131
1,Accountant / Auditor,men,1860,1805,0.000214
2,Accountant / Auditor,men,1870,1310,0.000100
3,Accountant / Auditor,men,1880,2295,0.000125
4,Accountant / Auditor,men,1900,11753,0.000396
...,...,...,...,...,...
7645,Welder,women,1960,28594,0.000329
7646,Welder,women,1970,48083,0.000440
7647,Welder,women,1980,57920,0.000451
7648,Welder,women,1990,36224,0.000243


In [ ]:
lista_colunas = df.columns.tolist()

In [118]:
# Função para salvar o gráfico como base64
def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png")
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    return img_str



def generate_html_with_jinja(output_file, context):
    # Configura o Jinja2 para carregar templates do diretório atual
    env = Environment(loader=FileSystemLoader('.'))

    # Carrega o template HTML
    template = env.get_template('template.html')

    # Renderiza o template com os dados (context)
    html_content = template.render(context)

    # Escreve o conteúdo renderizado em um arquivo HTML
    with open(output_file, "w") as file:
        file.write(html_content)



# Calcula a quantidade de valores ausentes (NaN) em cada coluna

missing_data = df.isnull().sum()

# Gera o gráfico de barras (histograma) da quantidade de valores ausentes por coluna

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=missing_data.index, y=missing_data.values, ax=ax)
ax.set_title('Quantidade de Dados Faltantes por Coluna')
ax.set_xlabel('Colunas')
ax.set_ylabel('Quantidade de Dados Faltantes')

# Converte o gráfico para base64 para ser incluído no HTML
plot_url = fig_to_base64(fig)
plt.close(fig)


context = {
    'title': 'Documento de Analise',
    'heading': 'Resultado da Analise de Dados',
    'description': 'Este documento contem a analise dos dados.',
    'data_results': [],
    'result_variables': [],
    'result_variables2': [],
    'plot_urls': plot_url,
    'info_df': [df.shape[1],df.shape[0],df.isnull().sum().sum(),df.duplicated().sum()]
}
# Para cada coluna, associar a média e a mediana
for item in lista_colunas:
  if pd.api.types.is_numeric_dtype(df[item]):
      media = df[item].mean()  # Cálculo da média
      mediana = df[item].median()  # Cálculo da mediana
      maximo = df[item].max()
      minimo = df[item].min()
      context['result_variables'].append({
          'variable': item,
          'mean': media,
          'median': mediana,
          'max': maximo,
          'min': minimo
      })

# Gera o arquivo HTML usando o template e os dados
generate_html_with_jinja("report_jinja.html", context)